Creates function that converts vector to column vector

In [1]:
import numpy as np
import pandas as pd

def as_colvec(x):   
    if (x.ndim == 2):
        return x
    else:
        return np.expand_dims(x,axis=1)

Creates 4d array

In [2]:
np.arange(4)

array([0, 1, 2, 3])

Creates a 4d column vector

In [3]:
as_colvec (np.arange(4))

array([[0],
       [1],
       [2],
       [3]])

Function that calculates Implied Returns

In [4]:
def implied_returns(delta,sigma,w):    #delta= risk aversion coefficient, sigma= var covar matrix, w=portfolio weights
    ir = delta* sigma.dot(w).squeeze()
    ir.name = 'Implied Returns'
    return ir

Black Litterman Functions

In [5]:
def proportional_prior(sigma,tau,p):
    helit_omega = p.dot(tau*sigma).dot(p.T)
    return pd.DataFrame(np.diag(np.diag(helit_omega.values)),index=p.index, columns=p.index)

In [6]:
from numpy.linalg import inv

def bl(w_prior, sigma_prior, p, q, 
       omega = None,
       delta=7.5, tau=.20):

    if omega is None:
         omega = proportional_prior(sigma_prior, tau, p)
    
    N = w_prior.shape[0]
    
    K = q.shape[0]
    
    pi = implied_returns(delta, sigma_prior, w_prior)
    
    sigma_prior_scaled = tau * sigma_prior
    
    nu_bl = pi + sigma_prior_scaled.dot(p.T).dot(inv(p.dot(sigma_prior_scaled).dot(p.T)+ omega).dot(q-p.dot(pi).values))
    ##nu_bl = pi + (p.T).dot(sigma_prior_scaled).dot(inv(sigma_prior_scaled.dot(p)).dot(p.T)+ omega).dot(q-p.dot(pi).values)
    #nu_bl = pi + (p.T)@(sigma_prior_scaled)@(inv(sigma_prior_scaled@(p))@(p.T)+ omega)@(q-p@(pi).values)

    
    sigma_bl = sigma_prior + sigma_prior_scaled - sigma_prior_scaled.dot(p.T).dot(inv(p.dot(sigma_prior_scaled).dot(p.T)+ omega)).dot(p).dot(sigma_prior_scaled)
    ##sigma_bl = sigma_prior + sigma_prior_scaled - (p.T).dot(sigma_prior_scaled).dot(inv(sigma_prior_scaled.dot(p).dot(p.T)+ omega)).dot(sigma_prior_scaled).dot(p)
    
    #sigma_bl = sigma_prior + sigma_prior_scaled - (p.T)@(sigma_prior_scaled)@(inv(sigma_prior_scaled@ p @(p.T)+ omega))@(sigma_prior_scaled)@p

    return (nu_bl, sigma_bl)


# Reads in Data

Reads only columns A:H from Excel i.e. until "Cash". Not the rest of the columns. 

It reads the prices and computes the returns of each product i.e. the percentage change between two successive prices.

In [36]:
prices = pd.read_excel('prices.xlsx',header=1,index_col=0, parse_dates= True, usecols="A:H")
returns = prices.pct_change()
returns = returns.dropna()

print(prices)

                     142.064800   144.349700   138.155125   128.191574   \
2004-08-13 00:00:00                                                       
2004-08-20              142.1387     144.3327   138.266540   128.319335   
2004-08-27              142.1587     144.4204   138.325071   128.933075   
2004-09-03              141.5786     143.8534   138.003604   129.354700   
2004-09-10              142.4513     144.6262   138.423808   129.771681   
2004-09-17              142.6912     144.9521   138.543642   130.486337   
2004-09-24              143.2086     145.3069   138.776881   130.970650   
2004-10-01              143.0084     145.1273   138.875262   130.794086   
2004-10-08              143.7116     145.8509   139.249365   131.284771   
2004-10-15              144.2088     146.3284   139.587171   131.617013   
2004-10-22              144.5268     146.6996   139.733671   131.563178   
2004-10-29              144.5377     146.7287   139.761422   132.087233   
2004-11-05              1

Computes the Covariance of returns and multiplies with 52 weeks to get year's covariance.

In [8]:
#returns= pd.DataFrame(returns)

#tickers = returns[:0] 
tickers = ['Euro Gov', 'Greek Gov', 'EU Corporate', 'EU HY' , 'Eur Eq', 'US Eq', 'Cash']
s = returns.cov()*52
s = np.matrix(s)
s = pd.DataFrame(s,index=tickers, columns=tickers)

Computes Implied Returns Weights

In [9]:

#input the weights of the benchmark
WEuroGov = .30
WGreekGov = .10
WEUCorporate = .20
WEUHY = .05
WEurEq = .10
WUSEq = .10
WCash = .15

weights = pd.Series([WEuroGov, WGreekGov, WEUCorporate, WEUHY, WEurEq, WUSEq, WCash],index= tickers)
#weights = np.matrix(weights)
#weights = pd.DataFrame(weights)

pi=implied_returns(delta=7.5,sigma=s, w=weights)
pi


Euro Gov        0.004326
Greek Gov       0.069389
EU Corporate    0.002424
EU HY           0.017458
Eur Eq          0.052837
US Eq           0.045285
Cash           -0.000062
Name: Implied Returns, dtype: float64

In [10]:
def inverse(d):
    
    return pd.DataFrame(inv(d.values), index=d.columns, columns=d.index)

def w_msr(sigma, mu, scale=True):
    
    w = inverse(sigma).dot(mu)
    
    if scale:
        w= w/sum(w)
        return w
    

In [11]:
#input the expectations
EEuroGov = .50
EGreekGov = .10
EEUCorporate = .20
EEUHY = .05
EEurEq = .05
EUSEq = .05
ECash = .0

mu_exp = pd.Series([EEuroGov, EGreekGov, EEUCorporate, EEUHY, EEurEq, EUSEq, ECash], index=tickers)
np.round(w_msr(s, mu_exp)*100,2)

Euro Gov        33.83
Greek Gov       -1.33
EU Corporate    74.64
EU HY          -10.24
Eur Eq          -0.03
US Eq            3.97
Cash            -0.83
dtype: float64

In [12]:
q= pd.Series(mu_exp)

p= pd.DataFrame([
    {'Euro Gov':1, 'Greek Gov':0, 'EU Corporate':0, 'EU HY':0, 'Eur Eq':0, 'US Eq':0, 'Cash':0},
    {'Euro Gov':0, 'Greek Gov':1, 'EU Corporate':0, 'EU HY':0, 'Eur Eq':0, 'US Eq':0, 'Cash':0},
    {'Euro Gov':0, 'Greek Gov':0, 'EU Corporate':1, 'EU HY':0, 'Eur Eq':0, 'US Eq':0, 'Cash':0},
    {'Euro Gov':0, 'Greek Gov':0, 'EU Corporate':0, 'EU HY':1, 'Eur Eq':0, 'US Eq':0, 'Cash':0},
    {'Euro Gov':0, 'Greek Gov':0, 'EU Corporate':0, 'EU HY':0, 'Eur Eq':1, 'US Eq':0, 'Cash':0},
    {'Euro Gov':0, 'Greek Gov':0, 'EU Corporate':0, 'EU HY':0, 'Eur Eq':0, 'US Eq':1, 'Cash':0},
    {'Euro Gov':0, 'Greek Gov':0, 'EU Corporate':0, 'EU HY':0, 'Eur Eq':0, 'US Eq':0, 'Cash':1}
    ])


In [13]:
bl_mu, bl_sigma = bl(w_prior= weights, sigma_prior=s, p=p, q=q)

bl_mu

Euro Gov        0.292083
Greek Gov       0.282988
EU Corporate    0.121205
EU HY           0.122930
Eur Eq          0.039985
US Eq          -0.049595
Cash           -0.000171
dtype: float64

In [14]:
w_msr(bl_sigma, bl_mu)

Euro Gov        0.331657
Greek Gov      -0.004294
EU Corporate    0.586814
EU HY          -0.023729
Eur Eq          0.002434
US Eq           0.009402
Cash            0.097717
dtype: float64